In [1]:
# !pip install gensim
# !pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download en_core_web_sm
# !pip install bert-extractive-summarizer --upgrade --force-reinstall
# !pip install spacy==2.1.3 --upgrade --force-reinstall
# !pip install -U nltk
# !pip install -U pywsd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.1 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/david/.local/share/virtualenvs/MCQ-StuDocu-Hackathon-2022-9n7-6BLn/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import nltk

nltk.download("stopwords")
nltk.download("popular")


[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/david/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/david/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/david/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/david/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/david/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package m

True





## BERT Extractive Summarizer
Summarize the text using BERT extractive summarizer. This is used to find important sentences and useful sentences from the complete text.

In [3]:
from summarizer import Summarizer

with open("egypt.txt", "r") as f:
    full_text = f.read()

model = Summarizer()
result = model(full_text, min_length=60, max_length=500, ratio=0.4)

summarized_text = "".join(result)


/home/david/.local/share/virtualenvs/MCQ-StuDocu-Hackathon-2022-9n7-6BLn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpo

## Keyword Extraction
Get important keywords from the text and filter those keywords that are present in the summarized text.

In [4]:
import itertools
import re
import pke
import string
from nltk.corpus import stopwords


def get_nouns_multipartite(text):
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text)
    #    not contain punctuation marks or stopwords as candidates.
    pos = {"PROPN"}
    # pos = {'VERB', 'ADJ', 'NOUN'}
    stoplist = list(string.punctuation)
    stoplist += ["-lrb-", "-rrb-", "-lcb-", "-rcb-", "-lsb-", "-rsb-"]
    stoplist += stopwords.words("english")
    # extractor.candidate_selection(pos=pos, stoplist=stoplist)
    extractor.candidate_selection(pos=pos)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1, threshold=0.75, method="average")
    keyphrases = extractor.get_n_best(n=20)

    return [key[0] for key in keyphrases]


keywords = get_nouns_multipartite(full_text)
filtered_keys = [
    keyword for keyword in keywords if keyword.lower() in summarized_text.lower()
]


## Sentence Mapping
For each keyword get the sentences from the summarized text containing that keyword. 

In [5]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor


def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences


def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key, values in keyword_sentences.items():
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences


sentences = tokenize_sentences(summarized_text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)


## Generate MCQ
Get distractors (wrong answer choices) from Wordnet/Conceptnet and generate MCQ Questions.

In [6]:
import requests
import json
import re
import random
from nltk.corpus import wordnet as wn
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
import conceptnet_lite
from conceptnet_lite import Label, edges_for


In [7]:
conceptnet_lite.connect()


NameError: name 'conceptnet_lite' is not defined

In [ ]:
# Distractors from Wordnet
def get_distractors_wordnet(syn, word):
    distractors = []
    word = word.lower()
    orig_word = word
    if len(word.split()) > 0:
        word = word.replace(" ", "_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0:
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        if name == orig_word:
            continue
        name = name.replace("_", " ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors


def get_wordsense(sent, word):
    word = word.lower()

    if len(word.split()) > 0:
        word = word.replace(" ", "_")

    if synsets := wn.synsets(word, "n"):
        wup = max_similarity(sent, word, "wup", pos="n")
        adapted_lesk_output = adapted_lesk(sent, word, pos="n")
        lowest_index = min(synsets.index(wup), synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None


# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word = word
    if len(word.split()) > 0:
        word = word.replace(" ", "_")
    distractor_list = []

    try:
        obj = Label.get(text=word, language="en").concepts
        edges = edges_for(obj, same_language=True)
    except Exception:
        return distractor_list
    for edge in edges:
        try:
            link = edge.end.text
            obj2 = Label.get(text=link, language="en").concepts
            edges2 = edges_for(obj2, same_language=True)
            for edge in edges2:
                if edge.relation.name in [
                    "PartOf",
                    "SimilarTo",
                    "DistinctFrom",
                    "MannerOf",
                ]:
                    word2 = edge.start.text
                    if (
                        word2 not in distractor_list
                        and original_word.lower() != word2.lower()
                    ):
                        distractor_list.append(word2.replace("_", " "))
        except Exception:
            continue

    return distractor_list


key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    if wordsense := get_wordsense(keyword_sentence_mapping[keyword][0], keyword):
        distractors = get_distractors_wordnet(wordsense, keyword)
        if len(distractors) == 0:
            distractors = get_distractors_conceptnet(keyword)
    else:
        distractors = get_distractors_conceptnet(keyword)
    if len(distractors) != 0:
        key_distractor_list[keyword] = distractors



#############################################################################
NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. 
#############################################################################


1)  _______  cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.
	 a )   Bantu
	 b )   Angolan
	 c )   Egyptians
	 d )   Algerian

More options:  ['Basotho', 'Beninese', 'Berber', 'Black African', 'Burundian', 'Cameroonian', 'Carthaginian', 'Chadian', 'Chewa', 'Congolese', 'Djiboutian', 'Egyptian', 'Ethiopian', 'Eurafrican', 'Ewe', 'Fulani'] 


2) It combined the red  _______  of Lower Egypt with the white  _______  of Upper Egypt.
	 a )   Academic Degree
	 b )   Aliyah
	 c )   Crown
	 d )   Academy Award

More options:  ['Cachet', 'Citation', 'Decoration', 'Emmy', 'Letter', 'Mention', 'Nobel Prize', 'Pennant', 'Prix De Rome', 'Prix Goncourt',

In [ ]:
final_output = {"question_set": []}
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub(" _______ ", sentence)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    final_output["question_set"].append({"question": output, "choices": top4choices, "answer": each})


{'question_set': [{'question': ' _______  cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.', 'choices': ['Egyptians', 'Angolan', 'Bantu', 'Algerian'], 'answer': 'egyptians'}, {'question': 'It combined the red  _______  of Lower Egypt with the white  _______  of Upper Egypt.', 'choices': ['Aliyah', 'Academic Degree', 'Academy Award', 'Crown'], 'answer': 'crown'}, {'question': ' _______  says a king named Narmer united Upper and Lower Egypt.', 'choices': ['Fable', 'Love Story', 'Legend', 'Adventure Story'], 'answer': 'legend'}]}
